# Séance 5 — PyCaret pour prototypage rapide

## 🎯 Objectifs
- Prototyper des modèles rapidement avec PyCaret
- Comparer automatiquement plusieurs algorithmes
- Interpréter les résultats avec des visualisations
- Exporter et sauvegarder le meilleur modèle

---

## 📚 Introduction

**PyCaret** est une bibliothèque d'AutoML low-code qui simplifie le workflow de machine learning. Elle permet de comparer rapidement des dizaines de modèles et d'automatiser le preprocessing.

**Installation :**
```bash
pip install pycaret[full]
```

**Note**: PyCaret fonctionne mieux avec un environnement isolé (virtualenv ou conda).

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.classification import *
import warnings
warnings.filterwarnings('ignore')

# Configuration
np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 📊 1. Préparation des données

Nous allons créer un dataset de classification textuelle simplifiée avec des features numériques extraites.

In [ ]:
# Création d'un dataset avec features textuelles extraites
np.random.seed(42)
n_samples = 500

# Simuler deux types de documents: Technical vs General
# Technical: plus long, plus de mots rares, phrases complexes
technical_docs = {
    'text_length': np.random.normal(2000, 500, n_samples // 2).clip(500, 5000),
    'avg_word_length': np.random.normal(7, 1.5, n_samples // 2).clip(3, 15),
    'num_sentences': np.random.normal(50, 15, n_samples // 2).clip(10, 150),
    'rare_word_count': np.random.poisson(30, n_samples // 2),
    'technical_terms': np.random.poisson(25, n_samples // 2),
    'avg_sentence_length': np.random.normal(25, 5, n_samples // 2).clip(10, 50),
    'unique_word_ratio': np.random.uniform(0.5, 0.8, n_samples // 2),
    'punctuation_count': np.random.normal(100, 30, n_samples // 2).clip(20, 300),
    'category': ['Technical'] * (n_samples // 2)
}

# General: plus court, mots simples, phrases courtes
general_docs = {
    'text_length': np.random.normal(800, 300, n_samples // 2).clip(200, 2000),
    'avg_word_length': np.random.normal(5, 1, n_samples // 2).clip(3, 10),
    'num_sentences': np.random.normal(20, 8, n_samples // 2).clip(5, 60),
    'rare_word_count': np.random.poisson(8, n_samples // 2),
    'technical_terms': np.random.poisson(3, n_samples // 2),
    'avg_sentence_length': np.random.normal(15, 3, n_samples // 2).clip(8, 30),
    'unique_word_ratio': np.random.uniform(0.3, 0.6, n_samples // 2),
    'punctuation_count': np.random.normal(40, 15, n_samples // 2).clip(10, 100),
    'category': ['General'] * (n_samples // 2)
}

# Combiner et mélanger
df_tech = pd.DataFrame(technical_docs)
df_gen = pd.DataFrame(general_docs)
df = pd.concat([df_tech, df_gen], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Dataset créé avec {len(df)} documents")
print(f"\nDistribution des classes:")
print(df['category'].value_counts())
print("\nAperçu des données:")
df.head(10)

## 🔍 2. Analyse exploratoire

In [ ]:
# Statistiques descriptives par catégorie
print("=" * 80)
print("STATISTIQUES PAR CATÉGORIE")
print("=" * 80)
print(df.groupby('category').mean())

# Visualisation
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

features = ['text_length', 'avg_word_length', 'num_sentences', 
            'rare_word_count', 'technical_terms', 'avg_sentence_length']

for idx, feature in enumerate(features):
    for cat in df['category'].unique():
        data = df[df['category'] == cat][feature]
        axes[idx].hist(data, alpha=0.6, label=cat, bins=20)
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Fréquence')
    axes[idx].set_title(f'Distribution: {feature}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## ⚙️ 3. Configuration de PyCaret

La fonction `setup()` initialise l'environnement PyCaret et effectue automatiquement le preprocessing.

In [ ]:
# Configuration de PyCaret
clf = setup(
    data=df,
    target='category',
    session_id=42,
    train_size=0.8,
    normalize=True,  # Normalisation des features
    feature_selection=False,  # Pas de sélection automatique pour ce demo
    remove_multicollinearity=False,
    silent=True,  # Moins de output
    verbose=False
)

print("✅ PyCaret configuré avec succès!")

## 🏆 4. Comparaison automatique de modèles

PyCaret peut comparer automatiquement des dizaines de modèles de classification.

In [ ]:
# Comparer tous les modèles disponibles
print("Comparaison de tous les modèles (peut prendre quelques minutes)...\n")
best_models = compare_models(
    n_select=5,  # Sélectionner les 5 meilleurs
    sort='F1',  # Trier par F1-score
    verbose=False
)

print("\n✅ Comparaison terminée!")

## 🎯 5. Créer et entraîner le meilleur modèle

In [ ]:
# Créer le meilleur modèle (basé sur la comparaison)
# Par exemple, essayons plusieurs modèles populaires

# Logistic Regression
lr = create_model('lr', verbose=False)
print("Logistic Regression créé")

# Random Forest
rf = create_model('rf', verbose=False)
print("Random Forest créé")

# XGBoost
xgb = create_model('xgboost', verbose=False)
print("XGBoost créé")

# LightGBM
lgb = create_model('lightgbm', verbose=False)
print("LightGBM créé")

print("\n✅ Tous les modèles créés!")

## 🔧 6. Optimisation des hyperparamètres

In [ ]:
# Tuner le meilleur modèle (XGBoost par exemple)
print("Optimisation des hyperparamètres...\n")
tuned_xgb = tune_model(
    xgb,
    optimize='F1',
    n_iter=10,  # Nombre d'itérations
    verbose=False
)

print("\n✅ Modèle optimisé!")

## 📊 7. Visualisations et interprétation

In [ ]:
# Matrice de confusion
print("Matrice de confusion:")
plot_model(tuned_xgb, plot='confusion_matrix', display_format='streamlit')

In [ ]:
# Courbe ROC
print("Courbe ROC:")
plot_model(tuned_xgb, plot='auc', display_format='streamlit')

In [ ]:
# Importance des features
print("Importance des features:")
plot_model(tuned_xgb, plot='feature', display_format='streamlit')

In [ ]:
# Courbe Precision-Recall
print("Courbe Precision-Recall:")
plot_model(tuned_xgb, plot='pr', display_format='streamlit')

## 📈 8. Évaluation finale sur l'ensemble de test

In [ ]:
# Évaluation finale
print("Évaluation sur l'ensemble de test (hold-out):")
predictions = predict_model(tuned_xgb)
print(predictions.head(10))

## 🎭 9. Ensemble de modèles (optionnel)

Combiner plusieurs modèles pour améliorer les performances.

In [ ]:
# Créer un ensemble (blending)
print("Création d'un ensemble de modèles...\n")
blended = blend_models(
    estimator_list=[lr, rf, tuned_xgb],
    verbose=False
)

print("\n✅ Ensemble créé!")

## 🎓 10. Finalisation et export du modèle

In [ ]:
# Finaliser le modèle (entraîner sur toutes les données)
print("Finalisation du modèle (entraînement sur toutes les données)...")
final_model = finalize_model(tuned_xgb)
print("✅ Modèle finalisé!")

# Sauvegarder le modèle
print("\nSauvegarde du modèle...")
save_model(final_model, 'best_text_classifier')
print("✅ Modèle sauvegardé sous 'best_text_classifier.pkl'")

## 🔮 11. Charger et utiliser le modèle sauvegardé

In [ ]:
# Charger le modèle
loaded_model = load_model('best_text_classifier')
print("✅ Modèle chargé!")

# Prédictions sur de nouvelles données
new_data = pd.DataFrame({
    'text_length': [2500, 600, 1800],
    'avg_word_length': [8, 5, 7],
    'num_sentences': [60, 15, 45],
    'rare_word_count': [35, 5, 20],
    'technical_terms': [30, 2, 15],
    'avg_sentence_length': [28, 12, 22],
    'unique_word_ratio': [0.7, 0.4, 0.6],
    'punctuation_count': [120, 30, 80]
})

print("\nPrédictions sur nouvelles données:")
predictions_new = predict_model(loaded_model, data=new_data)
print(predictions_new[['prediction_label', 'prediction_score']])

## 📋 12. Résumé des performances

In [ ]:
# Obtenir les métriques du modèle
print("=" * 80)
print("RÉSUMÉ DES PERFORMANCES DU MEILLEUR MODÈLE")
print("=" * 80)
print(f"\nModèle: {type(final_model).__name__}")
print(f"\nMétriques sur ensemble de validation (CV):")
# Ces métriques ont été affichées lors du create_model et tune_model
print("Voir les résultats ci-dessus pour les métriques détaillées.")
print("\n✅ Modèle prêt pour la production!")

## 🎯 EXERCICE : Votre projet AutoML

**Objectif**: Utiliser PyCaret pour une tâche de classification ou régression textuelle.

**Instructions**:
1. Créez ou utilisez un dataset avec des features extraites de texte
2. Configurez PyCaret avec différentes options de preprocessing
3. Comparez tous les modèles disponibles
4. Optimisez le meilleur modèle
5. Créez des visualisations pour interpréter les résultats
6. Exportez le modèle final

**Options avancées à explorer**:
- Feature engineering automatique
- Sélection de features
- Détection d'outliers
- Stacking d'ensembles
- Calibration des probabilités

**Questions à répondre**:
- Quel modèle performe le mieux ?
- Quelles features sont les plus importantes ?
- Y a-t-il du surapprentissage ?
- Comment améliorer les performances ?
- Le modèle est-il prêt pour la production ?

In [ ]:
# VOTRE CODE ICI
# Exemple de structure:

# 1. Créer/charger vos données
# my_data = ...

# 2. Configuration
# clf = setup(data=my_data, target='target_column', ...)

# 3. Comparaison
# best = compare_models()

# 4. Optimisation
# tuned = tune_model(best)

# 5. Visualisations
# plot_model(tuned, plot='confusion_matrix')
# plot_model(tuned, plot='feature')

# 6. Export
# final = finalize_model(tuned)
# save_model(final, 'my_model')

## 📝 Conclusion

Dans cette séance, nous avons appris à :
- Utiliser PyCaret pour le prototypage rapide de modèles ML
- Comparer automatiquement des dizaines d'algorithmes
- Optimiser les hyperparamètres avec peu de code
- Créer des visualisations pour interpréter les modèles
- Exporter et réutiliser les modèles entraînés

**Avantages de PyCaret** :
1. **Rapidité**: prototypage en quelques lignes de code
2. **Automatisation**: preprocessing, feature engineering, tuning
3. **Comparaison**: teste automatiquement de nombreux modèles
4. **Visualisations**: interprétation facilitée
5. **Production**: export simple des modèles

**Limitations** :
- Moins de contrôle fin que scikit-learn pur
- Nécessite de bonnes features en entrée
- Peut être lent pour de très gros datasets
- Abstractions peuvent masquer certains problèmes

**Prochaines étapes** :
- Essayer PyCaret sur vos propres datasets
- Explorer les modules regression, clustering, anomaly detection
- Intégrer PyCaret dans un pipeline de production
- Combiner avec deep learning pour le texte brut

**Ressources supplémentaires** :
- Documentation PyCaret: https://pycaret.org/
- Tutoriels: https://pycaret.org/tutorial/
- GitHub: https://github.com/pycaret/pycaret